<a href="https://colab.research.google.com/github/1zpzz/QM2---school-shooting-/blob/main/Data_cleaning_and_Visualisation_for_ESSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning

Download xls files from United States Census Bereau, clean and combine them into a comprehensive datasets that contain PPE data from 2007-2022.

In [82]:
import requests
import pandas as pd
import os

def download_and_process_excel(url):
    filename = os.path.basename(url)
    response = requests.get(url)

    with open(filename, "wb") as file:
        file.write(response.content)


    df = pd.read_excel(
        filename,     # Use the correct filename
        sheet_name="20",
        skiprows=5,
        header=[0, 1]      # Use the next TWO rows as header rows
    )

    # Drop any completely empty rows or columns
    df.dropna(how="all", axis=0, inplace=True)
    df.dropna(how="all", axis=1, inplace=True)

    # Flatten the multi-level columns into a single level.
    df.columns = [
        "_".join(map(str, col)).strip()
        for col in df.columns
    ]

    #    Rename columns so they match desired format:
    #    - First column => "Geographic area"
    #    - "Percentage change1" => "PCT"
    def rename_col(col):
        if "Geographic area" in col:
            return "Geographic area"

        # Convert "Percentage change1" to "PCT"
        col = col.replace("Percentage change1", "PCT")

        return col

    df.columns = [rename_col(c) for c in df.columns]
    df['Geographic area'] = df['Geographic area'].str.rstrip('.')

    # remove the non-states data
    df = df.iloc[:52]

    return df


# download the data and save as dataframe
df1 = download_and_process_excel("https://www2.census.gov/programs-surveys/school-finances/tables/2022/secondary-education-finance/elsec22_sumtables.xls")
df2 = download_and_process_excel("https://www2.census.gov/programs-surveys/school-finances/tables/2017/secondary-education-finance/elsec17_sumtables.xls")
df3 = download_and_process_excel("https://www2.census.gov/programs-surveys/school-finances/tables/2012/secondary-education-finance/elsec12_sttables.xls")

drop the pct data, leave only PPCS data and combine three df into one.

In [83]:
df1_ppcs = df1.filter(regex=r"(Geographic area|_PPCS$)").iloc[:, :-1]
df2_ppcs = df2.filter(regex=r"(Geographic area|_PPCS$)").iloc[:, :-1]
df3_ppcs = df3.filter(regex=r"(Geographic area|_PPCS$)")
for df in [df1, df2, df3]:
    df["Geographic area"] = (
        df["Geographic area"]
        .str.strip()
    )
df_ppcs_2007_2022 = (
    df1_ppcs
    .merge(df2_ppcs, on="Geographic area", how="inner")
    .merge(df3_ppcs, on="Geographic area", how="inner")
)
df_ppcs_2007_2022

,Geographic area,2022_PPCS,2021_PPCS,2020_PPCS,2019_PPCS,2018_PPCS,2017_PPCS,2016_PPCS,2015_PPCS,2014_PPCS,2013_PPCS,2012_PPCS,2011_PPCS,2010_PPCS,2009_PPCS,2008_PPCS,2007_PPCS
0,Alabama,11819.433746,10683.119878,10116.286143,10076.588583,9696.181213,9510.735237,9242.677695,9127.926974,9027.694266,8755.232705,8562.064460,8812.699399,8880.743341,8870.002468,9103.362547,8390.619597
1,Alaska,20190.928400,19540.424418,18313.421756,18393.569176,17725.939250,17837.859828,17509.975316,20172.490536,18415.906357,18175.000572,17390.400012,16673.958270,15782.501047,15551.821309,14629.708170,12300.203484
2,Arizona,10314.739106,9611.158860,8784.965162,8625.472268,8254.797509,8004.118423,7613.006435,7489.497110,7527.765094,7207.751512,7558.920930,7665.841231,7848.084196,7813.274023,7607.743900,7196.303609
3,Arkansas,12159.097404,11266.293801,10344.883675,10387.818428,10138.680596,9966.555044,9845.568548,9693.797141,9615.729733,9394.451934,9410.624150,9353.383759,9143.212107,8711.923747,8541.253072,8283.574132
4,California,17049.221380,14985.254079,14031.300940,14034.028821,12142.715059,12144.775158,11495.363449,10466.533409,9594.761823,9219.509520,9182.894693,9148.747042,9374.707573,9657.491772,9863.393658,9152.388099
5,Colorado,13422.070936,12875.723630,11602.362916,11029.670093,10201.958382,9808.601320,9574.742414,9245.032786,8985.188926,8646.668931,8547.661068,8723.757594,8852.782860,8718.476394,9078.575756,8166.769512
6,Connecticut,24452.958463,22769.264592,21345.602100,21310.452354,20634.919343,19321.716115,18957.841335,18377.286956,17744.721982,16631.120947,16273.654371,15616.503060,14906.372668,14531.122183,13847.996474,12979.327189
7,Delaware,19356.747112,17447.594686,17234.538627,16314.693547,15638.628923,15301.681723,14713.367705,14120.431696,13937.665288,13832.885369,13864.572093,12684.979729,12382.697997,12257.217219,12253.169329,11828.964579
8,District of Columbia,27424.714712,24535.113035,22855.918071,22561.581575,22758.863188,21974.000248,20235.000828,19395.645109,18485.047394,19076.190045,17468.465642,18475.078622,18666.849040,16407.683111,14594.335894,14324.412131
9,Florida,11075.890951,10401.219398,9936.722694,9645.306245,9346.035261,9075.492917,8919.956657,8881.138604,8755.366832,8432.599995,8371.974442,8886.535979,8741.321616,8760.383170,9034.824695,8513.769042


run the following cell if you want a csv file for the dataset.

In [81]:
df_ppcs_2007_2022.to_csv("df_ppcs_2007_2022.csv", index=False)